In [4]:
import ai2thor
import copy
import time
import random
import ai2thor.controller
import datetime
import cv2
import os
import matplotlib.pyplot as plt
import os
import numpy as np
# from utils.mid_level_constants import  scene_start_cheating_init_pose
from stretch_jupyter_helper import ENV_ARGS, initialize_arm, only_reset_scene, transport_wrapper, ADITIONAL_ARM_ARGS, execute_command

screen_size=224

ENV_ARGS['width'] = screen_size
ENV_ARGS['height'] = screen_size
ENV_ARGS['agentMode']='stretch'
ENV_ARGS['commit_id']='0926269d868a505fc81e69b474b1bfc1d5c06231'

saved_image_folder = '/Users/kianae/Desktop/saved_stretch_images'
os.makedirs(saved_image_folder, exist_ok=True)

controller = ai2thor.controller.Controller(**ENV_ARGS, renderDepthImage=True, renderInstanceSegmentation=True)
controller._build.url

'http://s3-us-west-2.amazonaws.com/ai2-thor-public/builds/thor-OSXIntel64-0926269d868a505fc81e69b474b1bfc1d5c06231.zip'

In [5]:
def translate(action):
    translation = {
        'MoveArmHeightM': 'j',
        'MoveArmHeightP': 'u',
        'MoveArmZM': 'z',
        'MoveArmZP': 'w',
#         'MoveArmYM': '4',
#         'MoveArmYP': '3',
        'MoveAhead': 'm',
        'RotateRight': 'r',
        'RotateLeft': 'l',
        '':'',
        'Done':'q',
        'PickUp':'p', 
        'Finish': 'q',
        'Drop': 'd',
    }
    return translation[action]
def setup_thirdparty_camera(controller, camera_position):
    controller.step('Pass')
    if len(controller.last_event.third_party_camera_frames) > 0:
        controller.step('UpdateThirdPartyCamera',
            thirdPartyCameraId=0, # id is available in the metadata response
            rotation=camera_position['rotation'],
            position=camera_position['position']
            )
    else:
        controller.step('AddThirdPartyCamera', 
            rotation=camera_position['rotation'], 
            position=camera_position['position'],
            fieldOfView=100)
def visualize(controller, save=False):
    agent_rotation = {'x':0, 'y':225, 'z':0}
    agent_camera = controller.last_event.metadata['cameraPosition']
    agent_camera['y'] += 0.2
    agent_camera['z'] += 1
    agent_camera['x'] += 1
    camera_position = {
        'position': agent_camera,
        'rotation': 
            agent_rotation
    }
    controller.step('Pass')
    setup_thirdparty_camera(controller, camera_position)
    image = controller.last_event.frame
    third_view = controller.last_event.third_party_camera_frames[0]
    combined=np.concatenate([image, third_view], 0)
    
    imagename = datetime.datetime.now().strftime("%m_%d_%Y_%H_%M_%S.%f")
    if save:
        plt.imsave(os.path.join(saved_image_folder, imagename+'.png'), combined)
    else:
        plt.imshow(combined)
    
def run_action_sequence(controller, action_sequence, object_id=None, target_location=None, logger_number=0, translated=False, seq_num_start = 0):
    picked_up = False
    if translated:
        translated_sequence = action_sequence
    else:
        translated_sequence = [translate(action) for action in action_sequence if action != '']
    for (seq_number, seq) in enumerate(translated_sequence):
        execute_command(controller, seq,ADITIONAL_ARM_ARGS)
                
def manual_task(scene_name, logger_number =0, final=False, reverse=False):
    only_reset_scene(controller, scene_name)
    all_actions = []
    actions_ran_so_far = 0
    while(True):
        visualize(controller, False)
        action = input()
        if not reverse:
            action = translate(action)
        if action == 'q':
            break
        all_actions.append(action)
        
        run_action_sequence(controller, [action], 
                            logger_number=logger_number, translated=True, seq_num_start=actions_ran_so_far)
        actions_ran_so_far += 1
        plt.cla()
        plt.imshow(controller.last_event.frame)
        plt.show()
        
    print(scene_name)
    print(all_actions)


In [6]:
camera_position = {
    'position': {'x': -1.3581395149230957,
   'y': 1.8946828126907349,
   'z': 3.151395082473755},
    'rotation': {
        'x':0,
        'y':40,
        'z':0
    }
}
controller.reset('FloorPlan2')
setup_thirdparty_camera(controller, camera_position)
for i in range(100):
    random_number = random.random()
    action = random.choice(['m', 'r', 'l', 'lift', 'reach', 'p', 'd'])
    if action == 'lift':
        controller.step(
            action="MoveArmBase",
            y=random.random() - 0.2,
            speed=1,
            returnToStart=True,
            fixedDeltaTime=0.02
        )
    elif action == 'reach':
        controller.step(
            action="MoveArm",
            position=dict(x=0, y=0, z=random.random()),
            coordinateSpace="armBase",
            restrictMovement=False,
            speed=1,
            returnToStart=True,
            fixedDeltaTime=0.02
        )
    elif action == 'm':
        controller.step(
            action='MoveAhead',
        )
    elif action == 't':
        controller.step(
            action='RotateRight',
        )
    elif action == 'l':
        controller.step(
            action='RotateLeft',
        )
    elif action == 'p':
        controller.step(
            action='PickupObject',
        )
    elif action == 'd':
        controller.step(
            action='ReleaseObject',
        )

    visualize(controller, True)

In [83]:
controller.reset('FloorPlan2')
controller.step('RotateRight', degrees=225)
controller.step(action='MoveArm', position=dict(x=0, y=0.2, z=0.2))
first_location = copy.deepcopy(controller.last_event.metadata['arm']['joints'])
visualize(controller, save=True)
controller.step(action='MoveArm', position=dict(x=0, y=1.9, z=1))
second_location = copy.deepcopy(controller.last_event.metadata['arm']['joints'])
visualize(controller, save=True)


In [84]:
controller.reset('FloorPlan2')
controller.step('RotateRight', degrees=225)
controller.step(action='MoveArm', position=dict(x=0, y=0.99, z=0.99))
visualize(controller, save=True)

In [ ]:
manual_task('FloorPlan1', logger_number =0, final=False, reverse=True)

In [57]:
[x['name'] for x in first_location]

['stretch_robot_lift',
 'stretch_robot_arm_1',
 'stretch_robot_arm_2',
 'stretch_robot_arm_3',
 'stretch_robot_arm_4',
 'stretch_robot_arm_5',
 'stretch_robot_wrist_1',
 'stretch_robot_wrist_2']

In [66]:
([(first_location[i]['rootRelativePosition']['y'], second_location[i]['rootRelativePosition']['y']) for i in range(len(first_location))])

[(1.4901161193847656e-08, 0.0),
 (7.450580596923828e-08, 0.0),
 (7.450580596923828e-08, 0.0),
 (7.450580596923828e-08, 0.0),
 (7.450580596923828e-08, 0.0),
 (7.450580596923828e-08, 0.0),
 (0.007252171635627747, 0.007252216339111328),
 (-0.030750200152397156, -0.030750274658203125)]

In [67]:
([(first_location[i]['position']['y'], second_location[i]['position']['y']) for i in range(len(first_location))])

[(0.24778443574905396, 1.2927844524383545),
 (0.24778449535369873, 1.2927844524383545),
 (0.24778449535369873, 1.2927844524383545),
 (0.24778449535369873, 1.2927844524383545),
 (0.24778449535369873, 1.2927844524383545),
 (0.24778449535369873, 1.2927844524383545),
 (0.2550365924835205, 1.3000366687774658),
 (0.2170342206954956, 1.2620341777801514)]

In [68]:
([(first_location[i]['rootRelativePosition']['z'], second_location[i]['rootRelativePosition']['z']) for i in range(len(first_location))])

[(-1.0430814256778831e-07, -1.0430814256778831e-07),
 (-4.172325134277344e-07, -4.172325134277344e-07),
 (-4.172325134277344e-07, -4.76837158203125e-07),
 (-4.172325134277344e-07, -4.76837158203125e-07),
 (-4.172325134277344e-07, -5.364418029785156e-07),
 (-3.8743019104003906e-07, -5.960464477539062e-07),
 (0.08307120203971863, 0.08307099342346191),
 (0.08304527401924133, 0.0830451250076294)]

In [69]:
([(first_location[i]['position']['z'], second_location[i]['position']['z']) for i in range(len(first_location))])

[(2.108029365539551, 2.108029365539551),
 (1.9279218912124634, 1.9279218912124634),
 (1.918729305267334, 1.7761588096618652),
 (1.909536600112915, 1.6243956089019775),
 (1.9003440141677856, 1.472632646560669),
 (1.9109541177749634, 1.3406721353530884),
 (1.9696946144104004, 1.3994126319885254),
 (1.969632863998413, 1.399350881576538)]

In [70]:
([(first_location[i]['rootRelativePosition']['x'], second_location[i]['rootRelativePosition']['x']) for i in range(len(first_location))])

[(2.9802311729554276e-08, 2.9802311729554276e-08),
 (0.25471022725105286, 0.25471022725105286),
 (0.26771050691604614, 0.4693356156349182),
 (0.28071093559265137, 0.6839612126350403),
 (0.29371121525764465, 0.898586630821228),
 (0.27870631217956543, 1.0852067470550537),
 (0.27870625257492065, 1.0852068662643433),
 (0.2787676751613617, 1.085268259048462)]

In [71]:
([(first_location[i]['position']['x'], second_location[i]['position']['x']) for i in range(len(first_location))])

[(-1.082889437675476, -1.082889437675476),
 (-0.9027823805809021, -0.9027823805809021),
 (-0.8935897946357727, -0.7510193586349487),
 (-0.8843970894813538, -0.599256157875061),
 (-0.8752045035362244, -0.44749313592910767),
 (-0.8858145475387573, -0.3155328631401062),
 (-0.8270741105079651, -0.2567923069000244),
 (-0.8270490169525146, -0.256767213344574)]